In [1]:
import pandas as pd
import pulp

# Set starting values
ride_list = pd.read_csv('RideData.csv')
num_rides = ride_list.shape[0]
rides = list(range(num_rides))
min_times = 0 #minimum # times for each ride
max_times = 2 #maximum # times for each ride
time_between = 15 #how long to allow between rides
food_time = (2.5 * 60) #how long to allow for meals
hours_open = (12 * 60) #how many hours the park is open

ride_list.head()

,name,wait,ride,rating,my_min,my_max,my_rating
0,Astro Orbiter,31,1.50,4,NaN,NaN,NaN
1,Barnstormer,12,1.05,2,NaN,NaN,NaN
2,Big Thunder Mountain Railroad,25,3.42,5,NaN,NaN,NaN
3,Buzz Lightyear's Space Ranger Spin,26,4.05,4,NaN,NaN,NaN
4,Dumbo: the Flying Elephant,15,1.50,4,NaN,NaN,NaN


In [2]:
# Get park hours
hours_open = input("How many hours is the park open on the day you're going?")
hours_open = float(hours_open)
if hours_open < 0:
    hours_open = 8
elif hours_open > 24:
    hours_open = 12
hours_open = hours_open * 60

# Get user non-ride times
data = input("How much time do you need for meals (in hours)? ")
data = float(data)
if data < 0:
    data = 0
elif data > hours_open:
    data = 3

data2 = input("How much time do you need for bathroom breaks (in hours)? ")
data2 = float(data2)
if data2 < 0:
    data2 = 0.5
elif data2 > hours_open - data:
    data2 = 3

data3 = input("How much time do you want for shopping (in hours)? ")
data3 = float(data3)
if data3 < 0:
    data3 = 0
elif data3 > (hours_open - data - data2):
    data3 = 3

food_time = data + data2 + data3
food_time = food_time * 60

How many hours is the park open on the day you're going?12
How much time do you need for meals (in hours)? 1.5
How much time do you need for bathroom breaks (in hours)? .5
How much time do you want for shopping (in hours)? .5


In [3]:
# Get user ride information
data = input("Do you want to enter your own ride ratings and preferences (enter Y or N)?")
if data.startswith("N") or data.startswith("n"):
    for i in rides:
        # Save default values into user values
        ride_list.loc[i, 'my_rating'] = ride_list.loc[i,'rating']
        ride_list.loc[i,'my_min'] = min_times
        ride_list.loc[i,'my_max'] = max_times
else:
    # Gather the user's ratings and limits
    for i in rides:
        data = input("What is your rating for {} from 1-5 where 5 is best? ".format(ride_list.loc[i, 'name']))
        data = int(float(data))
        if data >= 0 and data <= 5:
            ride_list.loc[i,'my_rating'] = data
        else:
            data = input("Please re-enter your rating for ", ride_list.loc[i, 'name'],": ")

        data = input("What is the maximum number of times you want to ride it (enter 0 if you don't want to ride it at all)? ")
        data = int(float(data))
        if data < 0:
            ride_list.loc[i,'my_max'] = 0
        else:
            ride_list.loc[i,'my_max'] = data

        if ride_list.loc[i,"my_max"] > 0:
            # They actually do want to ride this ride, get a minimum number of times
            data = input("What is the minimum number of times you want to ride it (enter 0 if it's ok if you don't ride it)? ")
            data = int(float(data))
            if data < 0:
                ride_list.loc[i, 'my_min'] = 0
            else:
                ride_list.loc[i, 'my_min'] = data
        else:
            ride_list.loc[i, 'my_min'] = 0
        print()

Do you want to enter your own ride ratings and preferences (enter Y or N)?y
What is your rating for Astro Orbiter from 1-5 where 5 is best? 1
What is the maximum number of times you want to ride it (enter 0 if you don't want to ride it at all)? 0

What is your rating for Barnstormer from 1-5 where 5 is best? 3
What is the maximum number of times you want to ride it (enter 0 if you don't want to ride it at all)? 2
What is the minimum number of times you want to ride it (enter 0 if it's ok if you don't ride it)? 0

What is your rating for Big Thunder Mountain Railroad from 1-5 where 5 is best? 5
What is the maximum number of times you want to ride it (enter 0 if you don't want to ride it at all)? 3
What is the minimum number of times you want to ride it (enter 0 if it's ok if you don't ride it)? 1

What is your rating for Buzz Lightyear's Space Ranger Spin from 1-5 where 5 is best? 2
What is the maximum number of times you want to ride it (enter 0 if you don't want to ride it at all)? 1


In [4]:
# Initialize model
model = pulp.LpProblem('Maximimize_ride_enjoyment', pulp.LpMaximize)

# Create variables to be adjusted in model
final_list = pulp.LpVariable.dicts("num", range(ride_list.shape[0]), cat = 'Integer')

# Add constraint to # times for each ride
for i in rides:
    model += pulp.LpConstraint(final_list[i])>=ride_list.loc[i, 'my_min']
    model += pulp.LpConstraint(final_list[i])<=ride_list.loc[i, 'my_max']
    
# Add calculation to be optimized
model += pulp.lpSum([(ride_list.loc[i,'ride'] * ride_list.loc[i,'rating'] * final_list[i])
                    for i in rides])

# Add constraint for total amount of time (without time between rides)
avail_time = hours_open - food_time
model += pulp.LpConstraint(# Total amount of time waiting & riding
                           ((pulp.lpSum([((ride_list.loc[i,'wait'] + ride_list.loc[i,'ride']) * 
                                         final_list[i]) for i in rides])) +
                           # Time traveling between rides
                           ((pulp.lpSum([final_list[i] for i in rides]) - 1) * time_between)) <=
                           # Less than or equal total time available 
                           avail_time)

# View model before solving
model

Maximimize_ride_enjoyment:
MAXIMIZE
6.0*num_0 + 2.1*num_1 + 13.75*num_10 + 42.5*num_11 + 2.0*num_12 + 12.5*num_13 + 12.5*num_14 + 55.0*num_15 + 9.39*num_16 + 9.5*num_17 + 10.07*num_18 + 18.75*num_19 + 17.1*num_2 + 16.2*num_3 + 6.0*num_4 + 37.5*num_5 + 31.5*num_6 + 45.4*num_7 + 1.5*num_8 + 3.0*num_9 + 0.0
SUBJECT TO
_C1: num_0 >= 0

_C2: num_0 <= 0

_C3: num_1 >= 0

_C4: num_1 <= 2

_C5: num_2 >= 1

_C6: num_2 <= 3

_C7: num_3 >= 0

_C8: num_3 <= 1

_C9: num_4 >= 0

_C10: num_4 <= 1

_C11: num_5 >= 1

_C12: num_5 <= 3

_C13: num_6 >= 0

_C14: num_6 <= 2

_C15: num_7 >= 0

_C16: num_7 <= 2

_C17: num_8 >= 0

_C18: num_8 <= 0

_C19: num_9 >= 0

_C20: num_9 <= 1

_C21: num_10 >= 1

_C22: num_10 <= 2

_C23: num_11 >= 1

_C24: num_11 <= 3

_C25: num_12 >= 0

_C26: num_12 <= 2

_C27: num_13 >= 0

_C28: num_13 <= 2

_C29: num_14 >= 0

_C30: num_14 <= 1

_C31: num_15 >= 0

_C32: num_15 <= 2

_C33: num_16 >= 0

_C34: num_16 <= 1

_C35: num_17 >= 0

_C36: num_17 <= 0

_C37: num_18 >= 1

_C38: num

In [5]:
# Solve model and check that it's optimized
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In [6]:
# Print out ride list for end user
for var in model.variables():
    # find i from var.name
    for key,value in final_list.items():
        if str(value) == str(var.name):
            i = key
    if var.varValue == 1:
        print("Ride",ride_list.loc[i,'name'],"once.")
    elif var.varValue > 1:
        print("Ride",ride_list.loc[i,'name'],"%1.0f" % var.varValue, "times.")

Ride Barnstormer once.
Ride Peter Pan's Flight once.
Ride Pirates of the Caribbean once.
Ride The Many Adventures of Winnie the Pooh once.
Ride Tomorrowland Transit Authority People Mover once.
Ride Big Thunder Mountain Railroad once.
Ride Haunted Mansion 3 times.
Ride It's a Small World once.
Ride Jungle Cruise once.
